# Course 4 : Explosion model with high order methods 

In [ ]:
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import root, fsolve
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"
import warnings
warnings.filterwarnings('ignore')

## One equation case

$$
d_{\tau} \widetilde{\theta} = f(\widetilde{\theta}) = \exp\Bigg(\frac{\widetilde{\theta}}{1 + \widetilde{\theta} \, / \, \beta}\Bigg) \Bigg(1 - \frac{\widetilde{\theta}}{\overline T_r} \Bigg)
$$

In [ ]:
class explosion_with_consumption_1eq_model:

    def __init__(self, Tr, beta):
        self.Tr = Tr
        self.beta = beta 

    def fcn(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        theta_dot = np.exp(theta/(1+(theta/beta))) * (1-theta/Tr) 
        return theta_dot
    
    def jac(self, t, theta):
        beta = self.beta
        Tr = self.Tr
        exp = np.exp(theta/(1+(theta/beta)))
        return (exp/(1+theta/beta)**2) * (1-theta/Tr) - exp/Tr

## Source term

In [ ]:
Tr = 200.0

theta = np.linspace(0, Tr, 1000)

fig = make_subplots(rows=2, cols=1, vertical_spacing=0.2,
                    subplot_titles=("Non-linearity of the source term", "Derivative of source term"))

beta = np.arange(10, 100, 10)
for i, beta_i in enumerate(beta):
    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta_i)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac
    fig.add_trace(go.Scatter(visible=False, x=theta, y=fcn(0, theta), name='f'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=theta, y=jac(0, theta), name='df/d\u03B8'), row=2, col=1)
    
fig.data[0].visible = True
fig.data[1].visible = True

# create slider
steps = []
for i, beta_i in enumerate(beta):
    step = dict(method="update", label = f"{beta_i}", 
                args=[{"visible": [(el==2*i) or (el==2*i+1) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(currentvalue={'prefix': '\u03B2 = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=800)
fig.update_xaxes(title_text="\u03B8", row=1, col=1)
fig.update_yaxes(title_text="f(\u03B8)", row=1, col=1)
fig.update_xaxes(title_text="\u03B8", row=2, col=1)
fig.update_yaxes(title_text="df/d\u03B8", row=2, col=1)
fig['layout']['sliders'][0]['pad']=dict(t= 50)
fig.show()

## One order method¶

In [ ]:
#####################################################
class ode_result:
    def __init__(self, y, t, rho=0):
        self.y = y
        self.t = t
        self.rho = rho

#####################################################
def forward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    #y = np.zeros((neq, nt), order='F')
    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y[:,it+1] = yn + dt*np.array(fcn(tn, yn))

    return ode_result(y, t)

#####################################################
def backward_euler(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    #y = np.zeros((neq, nt), order='F')
    y = np.zeros((neq, nt), order='F')
    y[:,0] = yini

    def g(uip1, *args):
        uip, tip1 = args
        return uip1 - uip - dt*np.array(fcn(tip1, uip1))

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        y0 = yn + dt*np.array(fcn(tn, yn))
        # solve y[:,it+1] - y[:,it] - dt * fcn(tini + (it+1)*dt, y[:,it+1]) = 0
        sol = root(g, y0, (yn, tn+dt))
        y[:,it+1] = sol.x

    return ode_result(y, t)

In [ ]:
Tr = 200.0
beta = 10. 

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0
nt = np.array([171, 201, 401, 501])

yini = (0.,)

fig =  make_subplots(rows=3, cols=1, vertical_spacing=0.1,
                     subplot_titles=("Solution (click on lengend to hide graph)", "Eigen value", "Eigen value . dt"))

for i, nt_i in enumerate(nt):
    
    dt = (tend-tini) / (nt_i-1)

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    sol_fe = forward_euler(tini, tend, nt_i, yini, fcn)
    sol_be = backward_euler(tini, tend, nt_i, yini, fcn)

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=sol_ref.y[0],  line_color='rgb(76,114,176)', legendgroup='sol', 
                             legendgrouptitle_text="Solution", name='ref sol'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=sol_fe.y[0], line_color='rgb(221,132,82)', legendgroup='sol', 
                             legendgrouptitle_text="Solution", name='forward euler'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=sol_be.y[0], line_color='rgb(85,168,104)', legendgroup='sol', 
                             legendgrouptitle_text="Solution", name='backward euler'), row=1, col=1)

    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size): eig_vals_ref[it] = jac(0, sol_ref.y[:,it])
    eig_vals_fe = np.zeros(sol_fe.t.size)
    for it in range(sol_be.t.size): eig_vals_fe[it] = jac(0, sol_fe.y[:,it])
    eig_vals_be = np.zeros(sol_be.t.size)
    for it in range(sol_be.t.size): eig_vals_be[it] = jac(0, sol_be.y[:,it])

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=eig_vals_ref, line_color='rgb(76,114,176)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='ref sol'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=eig_vals_fe, line_color='rgb(221,132,82)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='forward euler'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=eig_vals_be, line_color='rgb(85,168,104)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='backward_euler'), row=2, col=1)

    fig.add_trace(go.Scatter(visible=False, x=sol_fe.t, y=eig_vals_fe*dt, line_color='rgb(221,132,82)', legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt", name='forward euler'), row=3, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_be.t, y=eig_vals_be*dt, line_color='rgb(85,168,104)', legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt",  name='backward euler'), row=3, col=1)

for i in range(3*8, 4*8): fig.data[i].visible = True

# create slider
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}", 
                args=[{"visible": [(el>=8*i) and (el<=8*i+7) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(active=3, currentvalue={'prefix': 'nt = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=1000, legend_tracegroupgap=210, legend_groupclick="toggleitem")
fig.show()

## High order methods 

In [ ]:
############################################
def rk3(tini, tend, nt, yini, fcn):

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = np.array(fcn(tn, yn))
        k2 = np.array(fcn(tn + 0.5*dt, yn + dt*(0.5*k1)))
        k3 = np.array(fcn(tn + dt, yn + dt*(-k1 + 2*k2)))
        y[:,it+1] = yn + (dt/6)*(k1+4*k2+k3)

    return ode_result(y, t)

############################################
def rk4(tini, tend, nt, yini, fcn):
    
    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini

    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k1 = np.array(fcn(tn, yn))
        k2 = np.array(fcn(tn + 0.5*dt, yn + dt*(0.5*k1)))
        k3 = np.array(fcn(tn + 0.5*dt, yn + dt*(0.5*k2)))
        k4 = np.array(fcn(tn + dt, yn + dt*k3))
        y[:,it+1] = yn + (dt/6)*(k1+2*k2+2*k3+k4)

    return ode_result(y, t)

############################################
def radauIIA_3(tini, tend, nt, yini, fcn):
    
    c1 = 1/3
    c2 = 1.
    
    a11 = 5/12
    a12 = -1/12
    a21 = 3/4
    a22 = 1/4
    
    b1 = 3/4
    b2 = 1/4
    
    a = np.array([[a11, a12], [a21, a22]])
    rho = np.max(np.abs(np.linalg.eigvals(np.abs(a))))

    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini
    
    k0 = np.concatenate((fcn(tini,yini), fcn(tini,yini)))   
        
    def g(k, *args):        
        yn, tn = args
        k1, k2 = np.split(k,2)         
        fk1 = fcn(tn+c1*dt, yn+dt*(a11*k1+a12*k2))
        fk2 = fcn(tn+c2*dt, yn+dt*(a21*k1+a22*k2))
        return k - np.concatenate((fk1, fk2))     
    
    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k = fsolve(g, k0, (yn, tn))
        k0 = np.copy(k)
        k1, k2 = np.split(k,2)
        y[:,it+1] = yn + dt*(b1*k1+b2*k2)
        
    return ode_result(y, t, rho=rho)

############################################
def radauIIA_5(tini, tend, nt, yini, fcn):
    
    r6 = 6**0.5
    
    c1 = 2/5 - r6/10
    c2 = 2/5 + r6/10
    c3 = 1.
    
    a11 = 11/45 - 7*r6/360
    a12 = 37/225 - 169*r6/1800
    a13 = -2/225 + r6/75
    
    a21 = 37/225 + 169*r6/1800
    a22 = 11/45 + 7*r6/360
    a23 = -2/225 - r6/75
    
    a31 = 4/9 - r6/36
    a32 = 4/9 + r6/36
    a33 = 1/9

    b1 = 4/9 - r6/36
    b2 = 4/9 + r6/36
    b3 = 1/9
    
    a = np.array([[a11, a12, a13], [a21, a22, a23], [a31, a32, a33]])
    rho = np.max(np.abs(np.linalg.eigvals(np.abs(a))))
    
    dt = (tend-tini) / (nt-1)
    t = np.linspace(tini, tend, nt)

    yini = np.atleast_1d(yini)
    neq = yini.size

    y = np.zeros((neq, nt))
    y[:,0] = yini
    
    k0 = np.concatenate((fcn(tini,yini), fcn(tini,yini), fcn(tini,yini)))   

    def g(k, *args):        
        yn, tn = args
        k1, k2, k3 = np.split(k,3)         
        fk1 = fcn(tn+c1*dt, yn+dt*(a11*k1+a12*k2+a13*k3))
        fk2 = fcn(tn+c2*dt, yn+dt*(a21*k1+a22*k2+a23*k3))
        fk3 = fcn(tn+c3*dt, yn+dt*(a31*k1+a32*k2+a33*k3))
        return k - np.concatenate((fk1, fk2, fk3))     
    
    for it, tn  in enumerate(t[:-1]):
        yn = y[:,it]
        k = fsolve(g, k0, (yn, tn))
        k0 = np.copy(k)
        k1, k2, k3 = np.split(k,3)
        y[:,it+1] = yn + dt*(b1*k1+b2*k2+b3*k3)
        
    return ode_result(y, t, rho=rho)

In [ ]:
Tr = 200.0
beta = 10. 

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0
nt = np.array([81, 91, 101, 201])

yini = (0.,)

fig =  make_subplots(rows=3, cols=1, vertical_spacing=0.1,
                     subplot_titles=("Solution (click on lengend to hide graph)", "Eigen value", "Eigen value . dt . rho(A)"))

for i, nt_i in enumerate(nt):
    
    dt = (tend-tini) / (nt_i-1)

    emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
    fcn = emwc1eq.fcn
    jac = emwc1eq.jac

    sol_ref = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=1.e-8, rtol=1.e-8)
    sol_rk3 = rk3(tini, tend, nt_i, yini, fcn)
    sol_rk4 = rk4(tini, tend, nt_i, yini, fcn)
    sol_radau3 = radauIIA_3(tini, tend, nt_i, yini, fcn)
    sol_radau5 = radauIIA_5(tini, tend, nt_i, yini, fcn)

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=sol_ref.y[0],  line_color='rgb(76,114,176)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='ref sol'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_rk3.t, y=sol_rk3.y[0], line_color='rgb(221,132,82)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='RK3'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_rk4.t, y=sol_rk4.y[0], line_color='rgb(85,168,104)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='RK4'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_radau3.t, y=sol_radau3.y[0], line_color='rgb(221,132,82)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='Radau3IIA'), row=1, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_radau5.t, y=sol_radau5.y[0], line_color='rgb(129,114,179)', legendgroup='sol', 
                         legendgrouptitle_text="Solution", name='Radau5IIA'), row=1, col=1)

    eig_vals_ref = np.zeros(sol_ref.t.size)
    for it in range(sol_ref.t.size): eig_vals_ref[it] = jac(0, sol_ref.y[:,it])
    eig_vals_rk3 = np.zeros(sol_rk3.t.size)
    for it in range(sol_rk3.t.size): eig_vals_rk3[it] = jac(0, sol_rk3.y[:,it])
    eig_vals_rk4 = np.zeros(sol_rk4.t.size)
    for it in range(sol_rk4.t.size): eig_vals_rk4[it] = jac(0, sol_rk4.y[:,it])
    eig_vals_radau3 = np.zeros(sol_radau3.t.size)
    for it in range(sol_radau3.t.size): eig_vals_radau3[it] = jac(0, sol_radau3.y[:,it])
    eig_vals_radau5 = np.zeros(sol_radau5.t.size)
    for it in range(sol_radau5.t.size): eig_vals_radau5[it] = jac(0, sol_radau5.y[:,it])

    fig.add_trace(go.Scatter(visible=False, x=sol_ref.t, y=eig_vals_ref, line_color='rgb(76,114,176)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='ref sol'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_rk3.t, y=eig_vals_rk3, line_color='rgb(221,132,82)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='RK3'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_rk4.t, y=eig_vals_rk4, line_color='rgb(85,168,104)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='RK4'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_radau3.t, y=eig_vals_radau3, line_color='rgb(221,132,82)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='Radau3IIA'), row=2, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_radau5.t, y=eig_vals_radau5, line_color='rgb(129,114,179)', legendgroup='eig', 
                         legendgrouptitle_text="Eigen value", name='Radau5IIA'), row=2, col=1)

    rho3 = sol_radau3.rho
    rho5 = sol_radau5.rho
    
    fig.add_trace(go.Scatter(visible=False, x=sol_rk3.t, y=eig_vals_rk3*dt, line_color='rgb(221,132,82)', legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt", name='RK3'), row=3, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_rk4.t, y=eig_vals_rk4*dt, line_color='rgb(85,168,104)', legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt",  name='RK4'), row=3, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_radau3.t, y=eig_vals_radau3*dt*rho3, line_color='rgb(221,132,82)', legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt", name='Radau3IIA'), row=3, col=1)
    fig.add_trace(go.Scatter(visible=False, x=sol_radau5.t, y=eig_vals_radau5*dt*rho3, line_color='rgb(129,114,179)', legendgroup='eigdt', 
                         legendgrouptitle_text="Eigen value . dt",  name='Radau5IIA'), row=3, col=1)

for i in range(3*14, 4*14): fig.data[i].visible = True

# create sliderlen(fig.data)
steps = []
for i, nt_i in enumerate(nt):
    step = dict(method="update", label = f"{nt_i}", 
                args=[{"visible": [(el>=14*i) and (el<=14*i+13) for el in range(len(fig.data))]}])
    steps.append(step)
sliders = [dict(active=3, currentvalue={'prefix': 'nt = '}, steps=steps)]

fig.update_layout(sliders=sliders, height=1000, legend_tracegroupgap=210, legend_groupclick="toggleitem")
fig.show()

## Radau5 time steps

In [ ]:
Tr = 200.0
beta = 10.

emwc1eq = explosion_with_consumption_1eq_model(Tr, beta)
fcn = emwc1eq.fcn
jac = emwc1eq.jac

tini = 0.0
tend = 5.0
nt = 100

yini = (0.,)


tol_ref = 1e-9
sol_ref   = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=tol_ref, rtol=tol_ref)
dt_ref = sol_ref.t[1:]-sol_ref.t[:-1]
tol = 1e-4
sol_radau = solve_ivp(fcn, (tini, tend), yini, method='Radau', atol=tol, rtol=tol)
dt_radau = sol_radau.t[1:]-sol_radau.t[:-1]

print(f"Nb time steps for Radau with tol={tol_ref:.2e} (ref sol) : ", sol_ref.t.size - 1)
print(f"Nb time steps for Radau with tol={tol:.2e}           : ", sol_radau.t.size - 1)

fig =  make_subplots(rows=3, cols=1, vertical_spacing=0.1, 
                     subplot_titles=("Solution", f"dt Radau (tol={tol})", f"dt Radau (tol={tol_ref})"))

fig.add_trace(go.Scatter(x=sol_ref.t, y=sol_ref.y[0], name='ref sol'), row=1, col=1)
fig.add_trace(go.Scatter(x=sol_radau.t, y=sol_radau.y[0], name='radau'), row=1, col=1)

fig.add_trace(go.Bar(x=sol_radau.t[:-1]+0.5*dt_radau, y=dt_radau, width=dt_radau, name=f'tol={tol}', 
                       showlegend=False, marker_color='rgb(76,114,176)'), row=2, col=1)

fig.add_trace(go.Bar(x=sol_ref.t[:-1]+0.5*dt_ref, y=dt_ref, width=dt_ref, name='tol=1e-9', 
                     showlegend=False, marker_color='rgb(76,114,176)'), row=3, col=1)
                                                                           
fig.update_yaxes(type="log", exponentformat='e', row=2)
fig.update_yaxes(type="log", exponentformat='e', row=3)
fig.update_layout(height=1000)